In [7]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.chains import RetrievalQA

In [8]:
# =======================
# Step 1: Define Q&A Data
# =======================
qa_pairs = [
    {
        "question": "What services does our IT company provide?",
        "answer": "We provide AI and machine learning solutions, cloud infrastructure management, cybersecurity services, application development, and IT consulting tailored to enterprise clients."
    },
    {
        "question": "What is our company’s mission?",
        "answer": "Our mission is to help organizations accelerate digital transformation by delivering secure, scalable, and intelligent technology solutions."
    },
    {
        "question": "Where are our main offices located?",
        "answer": "Our headquarters is in London, with additional offices in New York, Bangalore, and Singapore."
    },
    {
        "question": "What industries do we serve?",
        "answer": "We serve clients across banking, healthcare, manufacturing, retail, and public sector industries."
    },
    {
        "question": "What is the process for onboarding a new client?",
        "answer": "The onboarding process includes requirement discovery, technical assessment, project scoping, contract finalization, and kickoff with the delivery team."
    },
    {
        "question": "Which cloud platforms do we work with?",
        "answer": "We work with Microsoft Azure, Amazon Web Services (AWS), and Google Cloud Platform (GCP)."
    },
    {
        "question": "Do we provide 24/7 technical support?",
        "answer": "Yes, we provide round-the-clock technical support and proactive monitoring for all managed services."
    },
    {
        "question": "What cybersecurity frameworks do we follow?",
        "answer": "We adhere to ISO 27001, NIST Cybersecurity Framework, and GDPR compliance standards."
    },
    {
        "question": "What programming languages are commonly used by our developers?",
        "answer": "Our developers primarily use Python, Java, C#, JavaScript, and Go, depending on project requirements."
    },
    {
        "question": "Do we offer AI consulting?",
        "answer": "Yes, we provide end-to-end AI consulting services, including strategy development, model design, and deployment."
    },
    {
        "question": "What are the typical project delivery timelines?",
        "answer": "Delivery timelines vary based on project scope, typically ranging from 6 weeks for MVPs to 6 months for enterprise deployments."
    },
    {
        "question": "What project management methodology do we use?",
        "answer": "We follow Agile methodologies, primarily Scrum and Kanban, for iterative and transparent delivery."
    },
    {
        "question": "Does the company support remote work?",
        "answer": "Yes, we support hybrid and remote work arrangements depending on the project and client needs."
    },
    {
        "question": "What is the typical team composition for a software project?",
        "answer": "A typical project team includes a project manager, solution architect, developers, QA engineers, and a DevOps specialist."
    },
    {
        "question": "What are our core company values?",
        "answer": "Our core values are innovation, integrity, collaboration, and customer success."
    },
    {
        "question": "How do we ensure data privacy for clients?",
        "answer": "We implement end-to-end encryption, role-based access control, and comply with international data protection regulations."
    },
    {
        "question": "Do we build mobile applications?",
        "answer": "Yes, we design and develop native and cross-platform mobile applications for Android and iOS using technologies like React Native and Flutter."
    },
    {
        "question": "What is our policy for software maintenance?",
        "answer": "We offer tiered maintenance plans covering bug fixes, feature updates, and performance optimization."
    },
    {
        "question": "Do we provide training or documentation to clients?",
        "answer": "Yes, we provide comprehensive documentation, technical handover sessions, and optional user training workshops."
    },
    {
        "question": "How do clients request support?",
        "answer": "Clients can log tickets through our customer portal, email our support desk, or call the 24/7 helpline."
    },
    {
        "question": "What tools do we use for project collaboration?",
        "answer": "We use Jira for task management, Confluence for documentation, and Microsoft Teams or Slack for communication."
    },
    {
        "question": "Do we offer DevOps automation services?",
        "answer": "Yes, we provide CI/CD pipeline setup, container orchestration, and infrastructure as code (IaC) services."
    },
    {
        "question": "What is our approach to software quality assurance?",
        "answer": "We perform unit testing, integration testing, performance testing, and automated regression testing to ensure product reliability."
    },
    {
        "question": "Do we provide UX/UI design services?",
        "answer": "Yes, our UX/UI team creates user-centered designs focusing on accessibility, aesthetics, and usability."
    },
    {
        "question": "What databases do we work with?",
        "answer": "We work with SQL databases like PostgreSQL, MySQL, and SQL Server, and NoSQL databases like MongoDB and Cassandra."
    },
    {
        "question": "Do we assist in cloud migration?",
        "answer": "Yes, we offer complete cloud migration services including architecture assessment, workload migration, and cost optimization."
    },
    {
        "question": "What makes our IT solutions stand out?",
        "answer": "Our solutions combine advanced AI capabilities, cloud-native design, and a focus on scalability and security."
    },
    {
        "question": "How do we handle project security?",
        "answer": "We enforce code reviews, vulnerability scanning, and continuous monitoring throughout the development lifecycle."
    },
    {
        "question": "Do we have experience with AI integration?",
        "answer": "Yes, we’ve integrated AI-driven chatbots, recommendation systems, and predictive analytics into enterprise applications."
    },
    {
        "question": "How can potential clients contact us?",
        "answer": "Clients can contact us via our website’s contact form, email sales@ourcompany.com, or call our sales hotline."
    }
]


In [9]:
# ==================================
# Step 2: Create Documents for RAG
# ==================================
docs = [
    Document(
        page_content=f"Question: {qa['question']}\nAnswer: {qa['answer']}",
        metadata={"source": "company_FAQ"}
    )
    for qa in qa_pairs
]

In [10]:
# ==================================
# Step 3: Create Vector Store (FAISS)
# ==================================
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embedding=embeddings)

In [12]:
# Create retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [13]:
# ==================================
# Step 4: Create the LLM
# ==================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [ ]:
# ==================================
# Step 5: Create RAG Chain
# ==================================
# Step 1: Retrieve relevant docs
relevant_docs = retriever.get_relevant_documents(query)

# Step 2: Combine retrieved context
context = "\n\n".join([doc.page_content for doc in relevant_docs])

# Step 3: Define LLM prompt
prompt = f"""
You are an intelligent company assistant.
Use the following context to answer the user's question accurately.

### CONTEXT:
{context}

### QUESTION:
{query}

### INSTRUCTIONS:
- Use only the information in the context.
- If the answer is not in the context, say "I don't have that information."
- Respond in a clear, concise, and professional tone.

### ANSWER:
"""

In [15]:
# ==================================
# Step 6: Ask a Question
# ==================================
query = "Do you offer cloud migration services?"
response = qa_chain.invoke({"query": query})

In [16]:

# ==================================
# Step 7: Display Results
# ==================================
print("=== QUESTION ===")
print(query)
print("\n=== ANSWER ===")
print(response["result"])
print("\n=== SOURCES ===")
for doc in response["source_documents"]:
    print(doc.metadata)


=== QUESTION ===
Do you offer cloud migration services?

=== ANSWER ===
Yes, we offer complete cloud migration services including architecture assessment, workload migration, and cost optimization.

=== SOURCES ===
{'source': 'company_FAQ'}
{'source': 'company_FAQ'}
{'source': 'company_FAQ'}
